# Open-Neural-APC as Tensorflow 2 implementation

In [1]:
# install and update required packages
!pip install --upgrade pip -q
# if you use tensorflow in a docker container:
#   comment 'tensorflow' from the 'requirements.txt' with a '#' otherwise cudnn lstm won't be available
!pip install --upgrade -r requirements.txt -q

# just for the video output/investigation (not necessary for training/testing)
!pip install --upgrade -r optionals.txt -q
# this is also optional, since it is needed for opencv
!apt-get -qq update && apt-get -qq install -y libsm6 libxext6 libxrender1 libfontconfig1

In [2]:
from utils import loadConfig, allow_growth
from tqdm import tqdm
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = loadConfig()
# Since I saved the data as "uint8" and the sensor is usually placed at a height of 2 meters
#    the resolution should be just below 1cm
#    but since the noise frames at the end of most sequences produce larger values, this might not be the case
# The "accuracy error niveau" is the absolutely permissible difference so that a prediction 
#    on the label is still counted as correct (remember: it's a regression task)
# The "jump input frames" parameter indicates which frames are used from the original sequence
# The original sequences are at about 40 FPS. The model is trained with just 10 FPS
# The "pretrain" parameter is not used so far but could be utilized with a pretraining of the input layer
# The "safe steps" parameter was used for the non-jupyter version to safe the model every 5 epochs

# switching between the gpus
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# disable annoying tf warnings (retracing etc.)
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
# enable GPU memory growth
allow_growth()

{
  "data parameter": {
    "data directory": "./data/",
    "input scaling factor": 255,
    "labels dtype": "uint32",
    "sequence dtype": "uint8",
    "training label": "train.csv",
    "validation label": "valid.csv"
  },
  "model parameter": {
    "input dimensions": [
      20,
      25
    ],
    "lstm depth": 5,
    "lstm width": 50,
    "output dimensions": 2
  },
  "training parameter": {
    "accuracy error niveau": 0.5,
    "aux scale": 3,
    "batch size": 16,
    "calculation dtype": "float32",
    "calculation epsilon": 1e-07,
    "dropout rate": 0.2,
    "epochs": 10000,
    "jump input frames": 4,
    "learning rate": 0.001,
    "maximum concatenation": 5,
    "minimum concatenation": 5,
    "optimizer clip parameter": [
      1.0,
      2.0
    ],
    "optimizer parameter": [
      0.9,
      0.999,
      1e-07
    ],
    "pretrain": false,
    "restrict dataset size": 1500,
    "safe steps": 5
  }
}


In [3]:
# enable effcient data processing
sequence_dtype = data_parameter["sequence dtype"]
labels_dtype = data_parameter["labels dtype"]
calculation_dtype = training_parameter["calculation dtype"]
calculation_epsilon = training_parameter["calculation epsilon"]

# enable single/half/double precision
import tensorflow.keras.backend as K
K.set_floatx(calculation_dtype)
K.set_epsilon(calculation_epsilon)

import numpy as np
# scale saved input into a normed range 
input_scaling_factor = np.asarray(data_parameter["input scaling factor"],dtype=calculation_dtype)

In [4]:
from napc import NeuralAPC

# First step into using multiple GPU's at once
# Needs adaption of batch size and learning rate (for optimal performance)
# The inference for video creation hat some problems with the distributed strategy
#mirrored_strategy = tf.distribute.MirroredStrategy()
#with mirrored_strategy.scope():

# This is the pure model. It's almost the same as in my bachelors thesis
# The only difference is an additional loss (auxiliary loss) and a dropout layer after the first dense layer
# Elsewise it's the same architecture, same optimizer, same 'main' loss
napc = NeuralAPC(model_parameter,training_parameter)
napc.compile()
napc.save()

# copy config into model folder
import shutil
config_path = shutil.copy2('config.json', napc.model_path)
print(f'Config saved under {config_path}')

Config saved under ./models/2020-06-05_23:34:38.708827/config.json


In [5]:
import os
import os.path as path
mode = "training"
data = path.join(data_parameter['data directory'], f'{mode}.dat')
data_lengths = path.join(data_parameter['data directory'], f'{mode}_meta.dat')

# This one solves the problem for all of you, which just click 'Run all' and expect it to work
# Since the training data is not uploaded 'on purpose', the code would usually break for most people
# If the file does not exists, the code immediately jumps to the point, where it loads the 'predefined' model and evaluates it
# Elsewise it trains the model and evalutes the newly trained
data_exists = os.path.isfile(data)

sequence_list = labels_list = None

if data_exists:
    from data_loader import readData
    # due to the expertise of github.com/xor2k, I switched from CSV to memory-mapped files.
    # This reduces the loading/mapping time by a lot.
    # The sequence_list is a list containing all sequences for training (The sequences have the shape of Tx20x25)
    # all sequences are differently long and are normalized in the range between 0. and 1., they have the dtype 'sequence_dtype'
    # The labels_list is a list with all the labels for training in the same order as the sequence_list is.
    # The shape of each label is 2 --> Therefore, the list could have the shape Nx2 as array. The labels have dtype 'labels_dtype'
    sequence_list, labels_list = readData(model_parameter,training_parameter,data,data_lengths,sequence_dtype,labels_dtype)
    # Similar to my bachelors thesis I used just about 1500 sequences (Not necessarily the same ones/ haven't checked it)
    # Not necessary for other people or experiments
    sequence_list, labels_list = sequence_list[:training_parameter["restrict dataset size"]], labels_list[:training_parameter["restrict dataset size"]]

Started reading files: 23:34:41 2020-06-05
Finished reading 1499 sequences. Took 1.440000 seconds.


In [6]:
from data_processing import Preprocess
preprocessor = None
if data_exists:
    # this is class which preprocesses the training data every epoch
    # it creates the necessary labels/bounds and augments the data
    preprocessor = Preprocess(sequence_list,labels_list,input_scaling_factor,training_parameter,calculation_dtype)

In [7]:
# train
if data_exists:
    
    # switching to 'better' generators in the near future
    # TODO: use/implement a more sophisticated generator
    def generator():
        for x,y in list(zip(*preprocessor.prepareEpoch())):
            yield x,y
    
    import tensorflow as tf
    
    # TODO: add custom callback for increasing the napc.epoch
    # TODO: callback which calls the napc.save() (or use the keras build-in callback)
    # TODO: add custom callback for creating the plots
    # TODO: add custom callback for creating the plots
    # TODO: either test the TF 2.2 profiler or use again a stop watch to see the slowest processes
    # TODO: Add tensorboard callback
    ds = tf.data.Dataset.from_generator(generator,(tf.float32, tf.float32),)
    napc.model.fit(ds,epochs=training_parameter['epochs'],initial_epoch=napc.epoch,\
                   max_queue_size=5, workers=3, use_multiprocessing=False)
    
    '''
    from matplotlib import pyplot as plt
    %matplotlib inline
    
    import time as t
    train_start = t.time()
    
    for epoch in tqdm(range(napc.epoch,training_parameter['epochs']),desc='Training progress'):
        start = t.time()

        X,Y = preprocessor.prepareEpoch()
        prep_end = t.time()
        
        for x,y in list(zip(X,Y)):
            metrics = napc.model.train_on_batch(x,y,reset_metrics=False)

        if epoch%100==0:
            print(','.join( np.vstack([['epoch'],[epoch]]).T.flatten() )+','+\
                  ','.join( np.vstack([['t_total','t_prep','t_calc'],\
                            [round(t.time()-train_start,2),round(prep_end-start,2),\
                             round(t.time()-prep_end,2)]]).T.flatten() )+','+\
                  ','.join( np.vstack([napc.model.metrics_names,np.round(metrics,5)]).T.flatten() ))

        napc.model.reset_metrics()
        napc.epoch += 1

        if epoch%1000==0:

            sample = 0
            x = x[sample:sample+1,:,:]
            y = y[sample:sample+1,:,:]

            prediction = napc.model.predict_on_batch(x)
            error = napc.loss_function(y[:,:,:2],y[:,:,2:4],prediction)

            # prepare plot
            num_plots = 2
            fig, ax = plt.subplots(1,num_plots,figsize=(14,3), dpi=80)
            fig.suptitle('Predictions in/out')

            for idx in range(num_plots):
                ax[idx].plot(y[0,:,idx],label='max')
                ax[idx].plot(y[0,:,idx+2],label='min')
                ax[idx].plot(y[0,:,4],label='end')
                ax[idx].plot(prediction[0,:,idx],label='prediction')
                ax[idx].plot(error[0,:,idx],label='error')
                ax[idx].legend()
            plt.show(fig)
    '''
    napc.save()
    # the output of this cell are metrics every 100th epoch and a plot of the counting behaviour every 1000th epoch
    # (*)the training took about 10 hours on a 2080TI while occupying less than 2GB VRAM and 80% GPU-Util.
    # * in the latest TF updates the GPU got significantly slower/more inefficient (even slower than a 75 Watt NVIDIA P2000)
    #   now using even less than 50% GPU utilization for the 2080TI while being slower

Epoch 1/10000
19/19 [==============================] - 7s 364ms/step - loss: 1.8265 - accuracy: 0.1411
Epoch 2/10000
19/19 [==============================] - 7s 378ms/step - loss: 1.7688 - accuracy: 0.1083
Epoch 3/10000
19/19 [==============================] - 7s 370ms/step - loss: 1.7573 - accuracy: 0.1169
Epoch 4/10000
19/19 [==============================] - 7s 362ms/step - loss: 1.7590 - accuracy: 0.1140
Epoch 5/10000
19/19 [==============================] - 6s 327ms/step - loss: 1.8041 - accuracy: 0.1223
Epoch 6/10000
19/19 [==============================] - 7s 368ms/step - loss: 1.6799 - accuracy: 0.1281
Epoch 7/10000
19/19 [==============================] - 7s 379ms/step - loss: 1.7526 - accuracy: 0.1180
Epoch 8/10000
19/19 [==============================] - 7s 361ms/step - loss: 1.8270 - accuracy: 0.1257
Epoch 9/10000
19/19 [==============================] - 6s 339ms/step - loss: 1.6843 - accuracy: 0.1232
Epoch 10/10000
19/19 [==============================] - 7s 352ms/step - l

In [8]:
# For all of you which don't have the training data and just want to execute it
if not data_exists:
    data_parameter, model_parameter, training_parameter = loadConfig('models/config.json')
    napc = NeuralAPC(model_parameter,training_parameter)
    # Loading the included model (it has no subdirectory)
    napc.loadModel(10000,'models/')
    # The model_path of the model is not 'models/', but the previously created subdirectory
    # You could now train it further/save it/ etc.

In [9]:
# produce videos on all validation sequences
import os.path as path
mode = "validation"
data = path.join(data_parameter['data directory'], f'{mode}.dat')
data_lengths = path.join(data_parameter['data directory'], f'{mode}_meta.dat')

# copy dict from training and modify the concatenation and batch size
validation_parameter = training_parameter.copy()
validation_parameter["minimum concatenation"] = 1
validation_parameter["maximum concatenation"] = 1
validation_parameter["batch size"]=1

# read the validation data
from data_loader import readData
sequence_list, labels_list = readData(model_parameter,validation_parameter,data,data_lengths,sequence_dtype,labels_dtype)

# process them (i need the bounds in y for the accuracy and the videos)
from data_processing import Preprocess
preprocessor = Preprocess(sequence_list,labels_list,input_scaling_factor,validation_parameter,calculation_dtype)
X,Y = preprocessor.prepareEpoch(training=False)

# create my videos
from optional_features import createVideo
accuracy = []
for idx,x,y in list(zip(range(len(X)),X,Y)):
    prediction = napc.model.predict_on_batch(x)
    createVideo(napc.epoch,idx,x,K.eval(prediction),y[:,:,0:2],y[:,:,2:4])
    accuracy += [K.eval(napc.accuracy(y,prediction))]

Started reading files: 20:04:10 2020-06-06
Finished reading 15 sequences. Took 0.000000 seconds.


Video progress: 534it [02:27,  3.61it/s]
Video progress: 141it [00:31,  4.41it/s]
Video progress: 146it [00:34,  4.27it/s]
Video progress: 504it [02:08,  3.91it/s]
Video progress: 131it [00:29,  4.50it/s]
Video progress: 146it [00:33,  4.34it/s]
Video progress: 147it [00:33,  4.38it/s]
Video progress: 116it [00:24,  4.67it/s]
Video progress: 177it [00:44,  3.98it/s]
Video progress: 630it [03:29,  3.01it/s]
Video progress: 151it [00:35,  4.24it/s]
Video progress: 223it [00:47,  4.66it/s]
Video progress: 137it [00:31,  4.41it/s]
Video progress: 293it [01:12,  4.06it/s]
Video progress: 146it [00:34,  4.24it/s]


In [10]:
# Since I'm not allowed to upload more sequences and I don't want to publish a perfect model
# (therefore, I haven't tested this one) the accuracy is just an approximation of the true capabilities
# The 'validation' data in this case is in fact a test set (last epoch was chosen without selection)
# In practice someone would use k-Fold-Crossvalidation and would reason about the average performance

# So let's have a look how well the model does...
print(f'{mode} accuracy: {100*np.mean(accuracy)} %')

validation accuracy: 93.33333373069763 %
